In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import faiss

features_path = Path("data") / "features"

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_33584\3544797485.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load all numpy files
features_list = [np.load(features_file) for features_file in sorted(features_path.glob("*.npy"))]

# Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(features_path / "features.npy", features)

In [3]:
# Normalize the features
features /= np.linalg.norm(features, axis=1)[:, np.newaxis]

# Create a Faiss index
d = features.shape[1]
index = faiss.IndexFlatIP(d)

# Add the features to the index
index.add(features)

In [4]:
# Load all the photo IDs
photo_ids = pd.concat([pd.read_csv(ids_file) for ids_file in sorted(features_path.glob("*.csv"))])
photo_ids.to_csv(features_path / "photo_ids.csv", index=False)